## Hcondever variable and frailty index

#### Gavin Qu, May 30th

In [1]:
import pandas as pd
import os

# Base directory containing the data files
base_dir = '/Users/gavinqu/Desktop/School/Dissertation/UKDA-6614-stata/stata/stata13_se/ukhls'

# List of base variable names to extract, including 'pidp'
base_hcond_variables = [
    'pidp', 'hcond1', 'hcond2', 'hcond3', 'hcond4', 'hcond5', 'hcond6', 'hcond7',
    'hcond8', 'hcond9', 'hcond10', 'hcond11', 'hcond12', 'hcond13', 'hcond14',
    'hcond15', 'hcond16', 'hcond17', 'hcond21', 'hcond22',
    'hcondn1', 'hcondn2', 'hcondn3', 'hcondn4', 'hcondn5', 'hcondn6', 'hcondn7',
    'hcondn8', 'hcondn9', 'hcondn10', 'hcondn11', 'hcondn12', 'hcondn13', 'hcondn14',
    'hcondn15', 'hcondn16', 'hcondn17', 'hcondnew1', 'hcondnew2',
    'hcondnew3', 'hcondnew4', 'hcondnew5', 'hcondnew6', 'hcondnew7', 'hcondnew8',
    'hcondnew10', 'hcondnew11', 'hcondnew12', 'hcondnew13', 'hcondnew14', 'hcondnew15',
    'hcondnew16'
]

base_disdif_variables = [f'disdif{i}' for i in range(1, 12)]

# Combine all variable names to extract
base_variables = ['pidp'] + base_hcond_variables[1:] + base_disdif_variables

# Wave prefixes from 'a' to 'm'
wave_prefixes = [chr(i) for i in range(ord('a'), ord('n'))]

# Function to load and filter wave data
def load_wave_data(wave_prefix, base_dir, base_variables):
    file_path = os.path.join(base_dir, f'{wave_prefix}_indresp.dta')
    if os.path.exists(file_path):
        print(f"Loading data from {file_path}")
        wave_data = pd.read_stata(file_path, convert_categoricals=False)
        
        # Construct the actual variable names for the current wave
        wave_variables = [f'{wave_prefix}_{var}' if var != 'pidp' else var for var in base_variables]
        age_column = f'{wave_prefix}_age_dv'
        wave_variables.append(age_column)
        
        # Find the intersection of desired variables and available columns
        available_columns = set(wave_variables).intersection(wave_data.columns)
        print(f"Available columns in {wave_prefix}: {available_columns}")
        
        # Select only the available columns
        if available_columns:
            selected_data = wave_data[list(available_columns)].copy()
            selected_data['wave'] = wave_prefix
            return selected_data
    return None

# List to store data from each wave
all_waves_data = []

# Loop through wave prefixes
for prefix in wave_prefixes:
    wave_data = load_wave_data(prefix, base_dir, base_variables)
    if wave_data is not None:
        all_waves_data.append(wave_data)

# Combine all waves into a single DataFrame
if all_waves_data:
    combined_data = pd.concat(all_waves_data, ignore_index=True)

    # Load the death information from xhhrel.dta
    death_file_path = os.path.join(base_dir, 'xhhrel.dta')
    death_data = pd.read_stata(death_file_path, convert_categoricals=False)

    # Select the pidp and death column
    death_data = death_data[['pidp', 'dcsedfl_dv']]

    # Merge the death information with the combined data
    combined_data = combined_data.merge(death_data, on='pidp', how='left')

    # List of columns to clean (excluding 'pidp', 'wave', 'dcsedfl_dv', and age columns)
    age_columns = [f'{prefix}_age_dv' for prefix in wave_prefixes]
    columns_to_clean = [col for col in combined_data.columns if col not in ['pidp', 'wave', 'dcsedfl_dv'] + age_columns]

    # Clean the data: treat everything that's not a 1 as 0
    def clean_data(df, columns):
        df_cleaned = df.copy()
        for col in columns:
            df_cleaned[col] = df_cleaned[col].apply(lambda x: 1 if x == 1 else 0)
        return df_cleaned

    # Apply the cleaning function
    cleaned_combined_data = clean_data(combined_data, columns_to_clean)

    # Create hcondever variables
    for i in range(1, 18):
        ever_var = f'hcondever{i}'
        cond_cols = [f'{prefix}_hcond{i}' for prefix in wave_prefixes] + [f'hcondn{i}', f'hcondnew{i}']
        cleaned_combined_data[ever_var] = cleaned_combined_data[cond_cols].max(axis=1)

    cleaned_combined_data['hcondever21'] = cleaned_combined_data[[f'{prefix}_hcond21' for prefix in wave_prefixes]].max(axis=1)
    cleaned_combined_data['hcondever22'] = cleaned_combined_data[[f'{prefix}_hcond22' for prefix in wave_prefixes]].max(axis=1)

    # Calculate the frailty index for each wave based on hcondever variables
    for prefix in wave_prefixes:
        relevant_columns = [f'hcondever{i}' for i in range(1, 18)] + ['hcondever21', 'hcondever22'] + [f'{prefix}_disdif{i}' for i in range(1, 12)]
        frailty_col = f'{prefix}_frailty'
        cleaned_combined_data[frailty_col] = cleaned_combined_data[relevant_columns].sum(axis=1) / len(relevant_columns)

    # Update age for subsequent waves
    for i, prefix in enumerate(wave_prefixes[1:], start=1):
        cleaned_combined_data[f'{prefix}_age_dv'] = cleaned_combined_data[f'a_age_dv'] + i

    # Display the first few rows of the cleaned combined data
    print("Cleaned Combined Data Head:")
    print(cleaned_combined_data.head())

    # Save the cleaned combined data to a new Stata file
    output_dir = '/Users/gavinqu/Desktop/School/Dissertation/EssexDissertation/Data'
    os.makedirs(output_dir, exist_ok=True)
    cleaned_combined_data_path = os.path.join(output_dir, 'cleaned_combined_ukhls_hcond_disdif_death_data.dta')
    cleaned_combined_data.to_stata(cleaned_combined_data_path, write_index=False)

    print(f"Cleaned combined data saved to {cleaned_combined_data_path}")
else:
    print("No data was loaded. Please check the file paths and variable names.")


Loading data from /Users/gavinqu/Desktop/School/Dissertation/UKDA-6614-stata/stata/stata13_se/ukhls/a_indresp.dta
Available columns in a: {'a_disdif6', 'a_hcond15', 'a_disdif9', 'a_hcond3', 'a_hcond6', 'a_hcond14', 'a_disdif3', 'a_hcond1', 'a_disdif8', 'a_disdif11', 'a_hcond8', 'a_disdif7', 'a_hcond2', 'a_hcond10', 'a_hcond16', 'a_hcond9', 'a_disdif2', 'a_disdif1', 'pidp', 'a_hcond12', 'a_hcond4', 'a_disdif5', 'a_hcond5', 'a_hcond7', 'a_disdif10', 'a_hcond13', 'a_hcond17', 'a_disdif4', 'a_age_dv', 'a_hcond11'}
Loading data from /Users/gavinqu/Desktop/School/Dissertation/UKDA-6614-stata/stata/stata13_se/ukhls/b_indresp.dta
Available columns in b: {'b_hcondn9', 'b_hcondn10', 'b_disdif9', 'b_hcondn2', 'b_hcondn5', 'b_disdif8', 'b_hcondn16', 'b_hcondn13', 'b_hcondn7', 'b_hcondn8', 'b_hcondn12', 'b_hcondn14', 'b_hcondn6', 'b_hcondn15', 'b_age_dv', 'b_disdif10', 'b_disdif6', 'b_disdif4', 'b_disdif7', 'pidp', 'b_disdif2', 'b_disdif3', 'b_hcondn1', 'b_hcondn17', 'b_disdif5', 'b_hcondn3', 'b_hc

/var/folders/n2/8hz3y3r90rj63gkzgrl1hwg40000gn/T/ipykernel_52456/2409186563.py:33: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  wave_data = pd.read_stata(file_path, convert_categoricals=False)


Available columns in c: {'c_hcond2', 'c_disdif7', 'c_hcondn5', 'c_hcondn15', 'c_disdif1', 'c_hcondn1', 'c_hcondn4', 'c_hcond5', 'c_hcond9', 'c_disdif10', 'c_hcondn17', 'c_hcondn14', 'c_hcond12', 'c_hcondn16', 'c_hcondn10', 'c_hcondn13', 'c_hcond3', 'c_hcondn3', 'pidp', 'c_hcondn2', 'c_hcondn8', 'c_hcondn7', 'c_hcondn11', 'c_hcond14', 'c_disdif2', 'c_hcond17', 'c_hcond16', 'c_disdif3', 'c_hcond11', 'c_hcond6', 'c_hcond4', 'c_hcondn6', 'c_disdif8', 'c_hcond7', 'c_hcond13', 'c_disdif5', 'c_disdif11', 'c_hcondn12', 'c_disdif4', 'c_hcond10', 'c_hcond15', 'c_hcond1', 'c_age_dv', 'c_disdif9', 'c_hcond8', 'c_disdif6', 'c_hcondn9'}
Loading data from /Users/gavinqu/Desktop/School/Dissertation/UKDA-6614-stata/stata/stata13_se/ukhls/d_indresp.dta
Available columns in d: {'d_hcond4', 'd_disdif8', 'd_hcond14', 'd_hcond5', 'd_hcondn4', 'd_disdif10', 'd_hcondn12', 'd_disdif4', 'd_hcondn14', 'd_disdif3', 'd_hcond16', 'd_hcond13', 'd_hcond17', 'd_hcondn10', 'd_hcondn3', 'd_hcond15', 'd_hcond12', 'd_disd

/var/folders/n2/8hz3y3r90rj63gkzgrl1hwg40000gn/T/ipykernel_52456/2409186563.py:33: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  wave_data = pd.read_stata(file_path, convert_categoricals=False)


Available columns in e: {'e_age_dv', 'e_hcondn5', 'e_hcondn7', 'e_hcond1', 'e_hcondn8', 'e_hcondn10', 'e_hcond6', 'e_hcond3', 'e_hcond14', 'e_disdif1', 'e_hcond16', 'e_hcondn14', 'e_hcond9', 'e_hcondn13', 'e_hcond4', 'e_hcond7', 'e_disdif9', 'e_hcondn16', 'e_disdif3', 'e_hcondn4', 'e_hcondn2', 'pidp', 'e_disdif6', 'e_hcondn11', 'e_hcondn9', 'e_disdif2', 'e_hcond15', 'e_hcond8', 'e_disdif5', 'e_hcond5', 'e_hcond17', 'e_hcondn1', 'e_hcondn12', 'e_hcondn17', 'e_disdif4', 'e_disdif11', 'e_hcond2', 'e_hcond12', 'e_hcondn3', 'e_disdif8', 'e_hcondn6', 'e_hcond13', 'e_disdif10', 'e_hcond10', 'e_hcondn15', 'e_hcond11', 'e_disdif7'}
Loading data from /Users/gavinqu/Desktop/School/Dissertation/UKDA-6614-stata/stata/stata13_se/ukhls/f_indresp.dta
Available columns in f: {'f_hcondn15', 'f_disdif8', 'f_hcond14', 'f_hcond9', 'f_disdif9', 'f_hcond11', 'f_hcondn14', 'f_hcondn11', 'f_disdif6', 'f_hcond16', 'f_hcondn7', 'f_disdif1', 'f_disdif11', 'f_hcond5', 'f_hcondn6', 'f_age_dv', 'f_disdif4', 'f_disdi